In [1]:
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import transforms


from captum.attr import (
    LRP,
    GuidedGradCam,
    InputXGradient,
    GuidedBackprop,
    IntegratedGradients,
    Saliency,
    KernelShap
)

In [4]:
import os
import urllib.request
from PIL import Image
from google.colab import files


In [5]:
%load_ext autoreload
%autoreload 1
%reload_ext autoreload

In [6]:
import drive.MyDrive.ijcnn_2023.pytorch_viz as viz
from matplotlib.colors import LinearSegmentedColormap

ModuleNotFoundError: ignored

In [ ]:
torch.manual_seed(123)
np.random.seed(123)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
'''
download image from net
'''
import urllib

## cat
url, filename = ("https://images.unsplash.com/photo-1495360010541-f48722b34f7d?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8NHx8Y2F0fGVufDB8fDB8fHww&auto=format&fit=crop&w=500&q=60",
                 "cat.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

## dog
url, filename = ("https://plus.unsplash.com/premium_photo-1666878155781-f86514e5808b?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MTE1fHxkb2d8ZW58MHx8MHx8fDA%3D&auto=format&fit=crop&w=500&q=60",
                 "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)


In [ ]:
img = Image.open(r"cat.jpg")
plt.imshow(img)
plt.show()
img = Image.open(r'dog.jpg')
plt.imshow(img)
plt.show()

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(Image.open('dog.jpg'))
input_batch = input_tensor.unsqueeze(0)

In [ ]:
'''
download three models: resnet18, inception_v3, and vgg11
further, download imagenet labels
'''

# Download models
model_resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model_inception = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model_vgg = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)

# Download ImageNet labels or get it  from the drive
# !wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
imagenet_classes = 'drive/MyDrive/ijcnn_2023/imagenet_classes.txt'

# Read the categories from the ImageNet
with open(imagenet_classes, "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [ ]:
model_resnet.eval()
with torch.no_grad():
    logits_resnet = model_resnet(input_batch)

In [ ]:
probs_resnet = nn.functional.softmax(logits_resnet[0], dim=0)

In [ ]:
viz.print_topk_categories(k=5, probs=probs_resnet, categories=categories)

In [ ]:
model_vgg.eval()
with torch.no_grad():
    logits_vgg = model_vgg(input_batch)

In [ ]:
probs_vgg = nn.functional.softmax(logits_vgg[0], dim=0)

In [ ]:
viz.print_topk_categories(k=5, probs=probs_vgg, categories=categories)

In [ ]:
model_inception.eval()
with torch.no_grad():
    logits_inception = model_inception(input_batch)

In [ ]:
probs_inception = nn.functional.softmax(logits_inception[0], dim=0)

In [ ]:
viz.print_topk_categories(k=5, probs=probs_inception, categories=categories)

---

## Lets start interpreting the models


1.   Using LRP
2.   Using Guided Grad CAM
3.   Using Input x Gradients
4.   Using Guided Backpropagation
5.   Integrated Gradients
6.   Saliency Map
7.   SHAP



In [ ]:
'''
create a baseline against which the model is going to get tallied.
here, we will create a baseline of all zeros
'''
baseline = torch.zeros(input_batch.shape)

In [ ]:
baseline.shape ## [1, 3, 224, 224] means one image with 3 channels and 224x224 resolution

**Layerwise Relevance Propagation**

In [ ]:
lrp_resnet = LRP(model_resnet)
attrs_lrp_resnet = lrp_resnet.attribute(input_batch, torch.topk(probs_resnet, 1).indices.item())

lrp_vgg = LRP(model_vgg)
attrs_lrp_vgg = lrp_vgg.attribute(input_batch, torch.topk(probs_vgg, 1).indices.item())

lrp_inception = LRP(model_inception)
attrs_lrp_inception = lrp_inception.attribute(input_batch, torch.topk(probs_inception, 1).indices.item())

**Guided Grad CAM**
(requires model and one of its layer as input)

In [ ]:
# model_resnet

In [ ]:
# model_vgg

In [ ]:
# model_inception

In [ ]:
gradcam_resnet = GuidedGradCam(model_resnet, model_resnet.layer4[1])
attrs_gradcam_resnet = gradcam_resnet.attribute(input_batch, torch.topk(probs_resnet, 1).indices.item())

gradcam_vgg = GuidedGradCam(model_vgg, model_vgg.features[20])
attrs_gradcam_vgg = gradcam_vgg.attribute(input_batch, torch.topk(probs_vgg, 1).indices.item())

gradcam_inception = GuidedGradCam(model_inception, model_inception.Mixed_7c.branch_pool.conv)
attrs_gradcam_inception = gradcam_inception.attribute(input_batch, torch.topk(probs_inception, 1).indices.item())

**Input x Grad**

In [ ]:
inpxgrad_resnet = InputXGradient(model_resnet)
attrs_inpxgrad_resnet = inpxgrad_resnet.attribute(input_batch, torch.topk(probs_resnet, 1).indices.item())

inpxgrad_vgg = InputXGradient(model_vgg)
attrs_inpxgrad_vgg = inpxgrad_vgg.attribute(input_batch, torch.topk(probs_vgg, 1).indices.item())

inpxgrad_inception = InputXGradient(model_inception)
attrs_inpxgrad_inception = inpxgrad_inception.attribute(input_batch, torch.topk(probs_inception, 1).indices.item())

**Guided Backprop**

In [ ]:
gbp_resnet = GuidedBackprop(model_resnet)
attrs_gbp_resnet = gbp_resnet.attribute(input_batch, torch.topk(probs_resnet, 1).indices.item())

gbp_vgg = GuidedBackprop(model_vgg)
attrs_gbp_vgg = gbp_vgg.attribute(input_batch, torch.topk(probs_vgg, 1).indices.item())

gbp_inception = GuidedBackprop(model_inception)
attrs_gbp_inception = gbp_inception.attribute(input_batch, torch.topk(probs_inception, 1).indices.item())

**Integrated Gradients**

In [ ]:
ig_resnet = IntegratedGradients(model_resnet)
attrs_ig_resnet = ig_resnet.attribute(input_batch, target=torch.topk(probs_resnet, 1).indices.item())

ig_vgg = IntegratedGradients(model_vgg)
attrs_ig_vgg = ig_vgg.attribute(input_batch, target=torch.topk(probs_vgg, 1).indices.item())

ig_inception = IntegratedGradients(model_inception)
attrs_ig_inception = ig_inception.attribute(input_batch, target=torch.topk(probs_inception, 1).indices.item())

**Saliency Maps**

In [ ]:
sal_resnet = Saliency(model_resnet)
attrs_sal_resnet = sal_resnet.attribute(input_batch, target=torch.topk(probs_resnet, 1).indices.item())

sal_vgg = Saliency(model_vgg)
attrs_sal_vgg = sal_vgg.attribute(input_batch, target=torch.topk(probs_vgg, 1).indices.item())

sal_inception = Saliency(model_inception)
attrs_sal_inception = sal_inception.attribute(input_batch, target=torch.topk(probs_inception, 1).indices.item())

**Kernel SHAP**

In [ ]:
shap_resnet = KernelShap(model_resnet)
attrs_shap_resnet = shap_resnet.attribute(input_batch, target=torch.topk(probs_resnet, 1).indices.item())

shap_vgg = KernelShap(model_vgg)
attrs_shap_vgg = shap_vgg.attribute(input_batch, target=torch.topk(probs_vgg, 1).indices.item())

shap_inception = KernelShap(model_inception)
attrs_shap_inception = shap_inception.attribute(input_batch, target=torch.topk(probs_inception, 1).indices.item())

---

# Now, time to VISUALIZE!!

In [ ]:
## define a custom heatmap (not necessary, can use predefined heatmap too)
## have a look at the original image and model predicitions

default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#ffffff'),
                                                  (0.25, '#000000'),
                                                  (1, '#000000')], N=256)

**Layerwise Relevance Propagation**

In [ ]:
viz.visualize_img(attrs_lrp_resnet, input_batch, default_cmap,  titles=['LRP on ResNet', ''])
viz.visualize_img(attrs_lrp_vgg, input_batch, default_cmap,  titles=['LRP on VGG', ''])
viz.visualize_img(attrs_lrp_inception, input_batch, default_cmap,  titles=['LRP on Inception', ''])

**Guided GradCam**

In [ ]:
viz.visualize_img(attrs_gradcam_resnet, input_batch, default_cmap,  titles=['Guided Grad CAM on ResNet', ''])
viz.visualize_img(attrs_gradcam_vgg, input_batch, default_cmap,  titles=['Guided Grad CAM on VGG', ''])
viz.visualize_img(attrs_gradcam_inception, input_batch, default_cmap,  titles=['Guided Grad CAM on Inception', ''])

**Input X Grad**

In [ ]:
viz.visualize_img(attrs_inpxgrad_resnet, input_batch, default_cmap, titles=['Input x Gradient on ResNet', ''])
viz.visualize_img(attrs_inpxgrad_vgg, input_batch, default_cmap, titles=['Input x Gradient on VGG', ''])
viz.visualize_img(attrs_inpxgrad_inception, input_batch, default_cmap, titles=['Input x Gradient on Inception', ''])

**Guided Backpropagation**

In [ ]:
viz.visualize_img(attrs_gbp_resnet, input_batch, default_cmap, titles=['Guided Backpropagation on ResNet', ''])
viz.visualize_img(attrs_gbp_vgg, input_batch, default_cmap, titles=['Guided Backpropagation on VGG', ''])
viz.visualize_img(attrs_gbp_inception, input_batch, default_cmap, titles=['Guided Backpropagation on Inception', ''])

**Integrated Gradients**

In [ ]:
viz.visualize_img(attrs_ig_resnet, input_batch, default_cmap, titles=['Integrated Gradients on ResNet', ''])
viz.visualize_img(attrs_ig_vgg, input_batch, default_cmap, titles=['Integrated Gradients on VGG', ''])
viz.visualize_img(attrs_ig_inception, input_batch, default_cmap, titles=['Integrated Gradients on Inception', ''])